# Cycling Potential Table

Data Extraction used to complement the paper data (regarding the distribution of cycling potential among modals and tiers)

In [1]:
%load_ext autoreload
%autoreload 2
import saopaulo.load_trips as sptr
import saopaulo.maps_aux as aux
import saopaulo.cycling_infrastructure as cinfra
import saopaulo.choropleth as spchoro

from shapely.geometry import LineString, Point
#import saopaulo.choropleth_folium as spchoro

import saopaulo.sp_grid as gr

from bikescience.intersect_ways import geometry_intersection_length

import geopandas as gpd
import json
import pandas as pd
import geopandas as gpd
import numpy as np
from ipywidgets import interact_manual, widgets, fixed
from IPython.core.display import display, HTML, clear_output
import folium

import warnings
import requests
import os
import fnmatch as fnm
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
od_trips = gpd.read_file('bases/routes_potential.shp')
display(od_trips)

,ID_ORDEM,ID_ORDEM.1,ID_ORDEM_1,orig_lat,orig_lon,dest_lat,dest_lon,ZONA,MUNI_DOM,CO_DOM_X,...,ID_ORDEM_2,length,modal,id,distance_p,age_potent,inclinatio,final_pote,final_po_1,geometry
0,1.0,1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,1.0,36.0,333743.0,...,1.0,1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366,0.428496,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
1,2.0,2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,1.0,36.0,333743.0,...,2.0,1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112,0.576615,"LINESTRING Z (-46.63515 -23.55155 783.14000, -..."
2,3.0,3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,1.0,36.0,333743.0,...,3.0,9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927,0.049649,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
3,4.0,4.0,4.0,-23.571829,-46.690238,-23.525949,-46.666559,1.0,36.0,333743.0,...,4.0,6428.546731,subway,4.0,0.077287,0.848482,0.402769,0.442846,0.240028,"LINESTRING Z (-46.69026 -23.57184 742.26000, -..."
4,4.0,4.0,4.0,-23.525949,-46.666559,-23.537903,-46.670921,1.0,36.0,333743.0,...,4.0,2667.571706,bus_sp,4.0,0.391186,0.848482,0.280026,0.506565,0.335606,"LINESTRING Z (-46.66683 -23.52593 733.11000, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180233,55728.0,55728.0,55728.0,-23.506304,-46.687070,-23.546946,-46.629749,138.0,36.0,327741.0,...,55728.0,0.100071,bus_sp,55728.0,0.056332,0.641112,0.830032,0.509159,0.443182,"LINESTRING Z (-46.68727 -23.50618 727.59000, -..."
180234,55728.0,55728.0,55728.0,-23.546946,-46.629749,-23.602704,-46.602333,138.0,36.0,327741.0,...,55728.0,0.074911,bus_sp,55728.0,0.056322,0.641112,0.429314,0.375582,0.242818,"LINESTRING Z (-46.62982 -23.54699 732.18000, -..."
180235,55728.0,55728.0,55728.0,-23.602704,-46.602333,-23.600440,-46.587185,138.0,36.0,327741.0,...,55728.0,0.022320,bus_sp,55728.0,0.056300,0.641112,0.925131,0.540847,0.490715,"LINESTRING Z (-46.60227 -23.60281 749.43000, -..."
180236,83593.0,83593.0,83593.0,-23.603440,-46.460983,-23.598433,-46.462309,231.0,36.0,355887.0,...,83593.0,0.008205,bus_sp,83593.0,0.039232,0.817337,0.993869,0.616813,0.516550,"LINESTRING Z (-46.46109 -23.60349 801.77000, -..."


In [3]:
def generate_potential_df(
    pedestrian, car, motorcycle, bus, subway, train,
    tier):
    
    trips = od_trips.loc[od_trips['final_po_1'] > tier * .25]
    trips = trips.loc[od_trips['final_po_1'] <= (tier + 1) * .25]
    
    if pedestrian:
        trips = trips.loc[trips['modal'] == 'pedestrian']
    if car:
        trips = trips.loc[(trips['modal'] == 'car') | 
                          (trips['modal'] == 'taxi') | 
                          (trips['modal'] == 'taxi_app')]
    if motorcycle:
        trips = trips.loc[trips['modal'] == 'motorcycle']
    if bus:
        trips = trips.loc[(trips['modal'] == 'bus_sp') | 
                          (trips['modal'] == 'bus_inter') | 
                          (trips['modal'] == 'bus_otherc') |
                          (trips['modal'] == 'bus_school') |
                          (trips['modal'] == 'bus_private')]
    if subway:
        trips = trips.loc[(trips['modal'] == 'subway') | 
                          (trips['modal'] == 'monorail')]
    if train:
        trips = trips.loc[trips['modal'] == 'train']
    return round(sum(trips['FE_VIA']))
    

In [4]:
import datetime
modals_name = ['pedestrian', 'car', 'motorcycle', 'bus', 'subway', 'train']

potential_matrix = [[0 for _ in range(4)] for _ in range(6)]
for tier in range(4):
    print('-----------------------------')
    print('Tier: ', tier)
    for modal in range(6):
        modals=[False, False, False, False, False, False]
        modals[modal] = True
        potentials = generate_potential_df(modals[0], modals[1], modals[2], modals[3], modals[4], modals[5], tier)
        potential_matrix[modal][tier] = potentials
        
        print('\t', modals_name[modal], ': ', potentials)
        
display(potential_matrix)

        

-----------------------------
Tier:  0
	 pedestrian :  140966
	 car :  4983205
	 motorcycle :  591425
	 bus :  8326401
	 subway :  1918556
	 train :  1773451
-----------------------------
Tier:  1
	 pedestrian :  1873679
	 car :  2502492
	 motorcycle :  189519
	 bus :  4553171
	 subway :  1092875
	 train :  430097
-----------------------------
Tier:  2
	 pedestrian :  7481648
	 car :  2865931
	 motorcycle :  202559
	 bus :  2690423
	 subway :  287914
	 train :  82710
-----------------------------
Tier:  3
	 pedestrian :  3847188
	 car :  1739362
	 motorcycle :  92631
	 bus :  1322868
	 subway :  133759
	 train :  21463


[[140966, 1873679, 7481648, 3847188],
 [4983205, 2502492, 2865931, 1739362],
 [591425, 189519, 202559, 92631],
 [8326401, 4553171, 2690423, 1322868],
 [1918556, 1092875, 287914, 133759],
 [1773451, 430097, 82710, 21463]]

In [5]:
pd.set_option('display.float_format', '{:.2f}'.format)
print(od_trips.groupby(['modal'])['FE_VIA'].sum())

modal
0                  591.86
bus_inter      1758115.00
bus_otherc     2970848.35
bus_private     351980.21
bus_school     2096602.53
bus_sp         9718944.53
car           11583312.69
monorail          8325.23
motorcycle     1076133.55
other           115141.73
pedestrian    13349875.82
subway         3426010.77
taxi            114303.73
taxi_app        393448.44
train          2310117.00
Name: FE_VIA, dtype: float64


In [21]:
car_trips = od_trips.loc[(od_trips['modal']=='car') | 
                         (od_trips['modal']=='taxi') | 
                         (od_trips['modal']=='taxi_app')]
print('car trips:                         ', round(sum(car_trips['FE_VIA'])))

high_potential_car = car_trips.loc[car_trips['final_po_1'] >= .75]
print('car trips high potential:          ', round(sum(high_potential_car['FE_VIA'])))

print('car trips high potential length:   ', round(sum(high_potential_car['FE_VIA'] * high_potential_car['length'])), 'm')

print('car trips high potential length mean:   ', round(sum(high_potential_car['FE_VIA'] * high_potential_car['length']) / sum(high_potential_car['FE_VIA'])), 'm')
print('car trips length mean:   ', round(sum(car_trips['FE_VIA'] * car_trips['length']) / sum(car_trips['FE_VIA'])), 'm')



car trips:                          12091065
car trips high potential:           1739437
car trips high potential length:    2426165863 m
car trips high potential length mean:    1395 m
car trips length mean:    7254 m


In [24]:
motorcycle_trips = od_trips.loc[od_trips['modal']=='motorcycle']
print('motorcycle trips:                         ', round(sum(motorcycle_trips['FE_VIA'])))

high_potential_motorcycle = motorcycle_trips.loc[motorcycle_trips['final_po_1'] >= .75]
print('motorcycle trips high potential:          ', round(sum(high_potential_motorcycle['FE_VIA'])))

print('motorcycle trips high potential length:   ', round(sum(high_potential_motorcycle['FE_VIA'] * high_potential_motorcycle['length'])), 'm')


print('car trips high potential length mean:   ', round(sum(high_potential_motorcycle['FE_VIA'] * high_potential_motorcycle['length']) / sum(high_potential_motorcycle['FE_VIA'])), 'm')

print('car trips length mean:   ', round(sum(motorcycle_trips['FE_VIA'] * motorcycle_trips['length']) / sum(motorcycle_trips['FE_VIA'])), 'm')

motorcycle trips:                          1076134
motorcycle trips high potential:           92631
motorcycle trips high potential length:    143610944 m
car trips high potential length mean:    1550 m
car trips length mean:    9881 m


In [26]:
# according to EEA (https://www.eea.europa.eu/ims/co2-performance-of-new-passenger)
# cars emissions are 107.5 gCO2/km on avg

cyclable_motorized_individual = od_trips.loc[(od_trips['modal']=='car') | 
                         (od_trips['modal']=='taxi') | 
                         (od_trips['modal']=='taxi_app') |
                         (od_trips['modal']=='motorcycle')].loc[
                         (od_trips['final_po_1']>=.75)]

length_motorized_individual = sum(cyclable_motorized_individual['FE_VIA'] * cyclable_motorized_individual['length'])

km = length_motorized_individual / 1000
gCO2 = km * 107.5

kmCO2 = gCO2 / 1000
print(round(km, 2))
print(round(kmCO2, 2))


2569776.81
276251.01
